In [1]:
# lstm_lstm: emotion recognition from speech= lstm, text=lstm
# created for ATSIT paper 2020
# coded by Bagus Tris Atmaja (bagus@ep.its.ac.id)

import numpy as np
import pickle
import pandas as pd

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, CuDNNLSTM, TimeDistributed, \
                         Bidirectional, Flatten, \
                         Embedding, Dropout, Flatten, BatchNormalization, \
                         RNN, concatenate, Activation, LSTM
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import random as rn
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers.convolutional import Conv2D

Using TensorFlow backend.


In [2]:
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)

In [3]:

# load feature and labels
feat = np.load('../../dimensionalSM/data/feat_34_hfs.npy')
vad = np.load('../../dimensionalSM/data/y_egemaps.npy')
# remove outlier, < 1, > 5
vad = np.where(vad==5.5, 5.0, vad)
vad = np.where(vad==0.5, 1.0, vad)

# standardization
scaled_feature = False

# text feature
#path = '/home/s1820002/IEMOCAP-Emotion-Detection/'
data = np.load('../../dimensionalSM/IEMOCAP_full_release/data_collected_10039.pickle', allow_pickle=True)
text = [t['transcription'] for t in data]
#vad = np.load('../../data/y_egemaps.npy')
# load fastext model
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)

token_tr_X = tokenizer.texts_to_sequences(text)
MAX_SEQUENCE_LENGTH = len(max(token_tr_X, key=len))
x_train_text = []
x_train_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)

word_index = tokenizer.word_index
nb_words = len(word_index) +1
print('Found %s unique tokens' % len(word_index))
#g_word_embedding_matrix = np.load(path+'g_word_embedding_matrix.npy')

# other parameters
# MAX_SEQUENCE_LENGTH = 554
EMBEDDING_DIM = 300
# nb_words = 3438

Found 3437 unique tokens


In [4]:
if scaled_feature == True:
    scaler = StandardScaler()
    scaler = scaler.fit(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaler.transform(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaled_feat.reshape(feat.shape[0], feat.shape[1], feat.shape[2])
    feat = scaled_feat
else:
    feat = feat

scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1, keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss

In [5]:
import pickle
with open('../../dimensionalSM/IEMOCAP_full_release/data_collected_333.pickle', 'rb') as handle:
    data2 = pickle.load(handle)
    
x_train_mocap = []
counter = 0
for ses_mod in data2:
    x_head = ses_mod['mocap_head']
    if(x_head.shape != (200,18)):
        x_head = np.zeros((200,18))   
    x_head[np.isnan(x_head)]=0
    x_hand = ses_mod['mocap_hand']
    if(x_hand.shape != (200,6)):
        x_hand = np.zeros((200,6))   
    x_hand[np.isnan(x_hand)]=0
    x_rot = ses_mod['mocap_rot']
    if(x_rot.shape != (200,165)):
        x_rot = np.zeros((200,165))   
    x_rot[np.isnan(x_rot)]=0
    x_mocap = np.concatenate((x_head, x_hand), axis=1)
    x_mocap = np.concatenate((x_mocap, x_rot), axis=1)
    x_train_mocap.append( x_mocap )
    
x_train_mocap = np.array(x_train_mocap)
x_train_mocap = x_train_mocap.reshape(-1,200,189,1)
x_train_mocap.shape

(10039, 200, 189, 1)

In [6]:
split = 7869

In [7]:
# API model
def model():
    # speech network
    input_speech = Input(shape=(feat.shape[1], feat.shape[2]), name='speech_input')
    net_speech = BatchNormalization()(input_speech)
    net_speech = LSTM(32, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(128, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(32, return_sequences=True)(net_speech)
    model_speech = Flatten()(net_speech)
    #model_speech = Dropout(0.1)(net_speech)
    
    #text network
    input_text = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    net_text = Embedding(nb_words,
                    EMBEDDING_DIM,
                    trainable = True)(input_text)
    net = Bidirectional(LSTM(128, return_sequences=True))(net_text)
    net = Bidirectional(LSTM(64, return_sequences=False))(net)  
    model_text = Dense(64)(net)
    
    #mocap network
    input_mocap = Input(shape=(200,189,1))
    model_mocap1 = Conv2D(64, 3, strides=(3, 3), padding="same")(input_mocap)
    model_mocap2 = Dropout(0.2)(model_mocap1)
    model_mocap3 = Activation('relu')(model_mocap2)
    model_mocap4 = Conv2D(128, 3,strides=(3, 3), padding="same")(model_mocap3)
    model_mocap5 = Dropout(0.2)(model_mocap4)
    model_mocap6 = Activation('relu')(model_mocap5)
    model_mocap7 = Conv2D(256, 3,strides=(3, 3), padding="same")(model_mocap6)
    model_mocap8 = Dropout(0.2)(model_mocap7) 
    model_mocap9 = Activation('relu')(model_mocap8)
    model_mocap10 = Conv2D(128, 3,strides=(3, 3), padding="same")(model_mocap9)   
    model_mocap11 = Dropout(0.2)(model_mocap10)  
    model_mocap12 = Activation('relu')(model_mocap11)
    model_mocap13 = Conv2D(64, 3,strides=(3, 3), padding="same")(model_mocap12)  
    model_mocap14 = Dropout(0.2)(model_mocap13) 
    model_mocap15 = Activation('relu')(model_mocap14)
    model_mocap = Flatten()(model_mocap15) 
    # combined model
    model_combined = concatenate([model_speech, model_text, model_mocap])
    model_combined = Dense(64, activation='relu')(model_combined)
    model_combined = Dense(32, activation='relu')(model_combined)
    #model_combined = Dropout(0.4)(model_combined)
    target_names = ('v', 'a', 'd')
    model_combined = [Dense(1, name=name)(model_combined) for name in target_names]

    model = Model([input_speech, input_text, input_mocap], model_combined) 
    model.compile(loss=ccc_loss,
                  loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model


#model_1.summary()

In [8]:
# main function for for LOSO
def main():
    model_1 = model()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model_1.fit( [feat[:split],x_train_text[:split],x_train_mocap[:split]],
                      vad[:split].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model_1.evaluate( [feat[split:],x_train_text[split:],x_train_mocap[split:]], vad[split:].T.tolist())
    print(metrik[-3:])
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.5, 0.8, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(30):
        best = main()
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('stmWE_multi7869-3.csv', header=['time', 'v','a','d', 'ave'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 79s 13ms/step - loss: 2.2943 - v_loss: 0.8085 - a_loss: 0.7139 - d_loss: 0.7703 - v_ccc: 0.1909 - a_ccc: 0.2857 - d_ccc: 0.2291 - val_loss: 1.9968 - val_v_loss: 0.7184 - val_a_loss: 0.6324 - val_d_loss: 0.6698 - val_v_ccc: 0.2868 - val_a_ccc: 0.3762 - val_d_ccc: 0.3402
Epoch 2/50
6295/6295 [==============================] - 73s 12ms/step - loss: 1.1368 - v_loss: 0.3758 - a_loss: 0.3298 - d_loss: 0.4307 - v_ccc: 0.6240 - a_ccc: 0.6701 - d_ccc: 0.5691 - val_loss: 1.6354 - val_v_loss: 0.5934 - val_a_loss: 0.4910 - val_d_loss: 0.5592 - val_v_ccc: 0.4069 - val_a_ccc: 0.5159 - val_d_ccc: 0.4418
Epoch 3/50
6295/6295 [==============================] - 73s 12ms/step - loss: 0.9817 - v_loss: 0.2911 - a_loss: 0.3056 - d_loss: 0.3846 - v_ccc: 0.7089 - a_ccc: 0.6943 - d_ccc: 0.6152 - val_loss: 1.5480 - val_v_loss: 0

Epoch 6/50
6295/6295 [==============================] - 70s 11ms/step - loss: 0.7620 - v_loss: 0.2001 - a_loss: 0.2655 - d_loss: 0.2960 - v_ccc: 0.7998 - a_ccc: 0.7344 - d_ccc: 0.7038 - val_loss: 1.5341 - val_v_loss: 0.5226 - val_a_loss: 0.4376 - val_d_loss: 0.5840 - val_v_ccc: 0.4798 - val_a_ccc: 0.5673 - val_d_ccc: 0.4189
Epoch 7/50
6295/6295 [==============================] - 70s 11ms/step - loss: 0.7151 - v_loss: 0.1882 - a_loss: 0.2472 - d_loss: 0.2806 - v_ccc: 0.8121 - a_ccc: 0.7531 - d_ccc: 0.7197 - val_loss: 1.5285 - val_v_loss: 0.5543 - val_a_loss: 0.4212 - val_d_loss: 0.5578 - val_v_ccc: 0.4439 - val_a_ccc: 0.5807 - val_d_ccc: 0.4469
Epoch 8/50
6295/6295 [==============================] - 70s 11ms/step - loss: 0.6748 - v_loss: 0.1766 - a_loss: 0.2341 - d_loss: 0.2645 - v_ccc: 0.8233 - a_ccc: 0.7660 - d_ccc: 0.7359 - val_loss: 1.5575 - val_v_loss: 0.5337 - val_a_loss: 0.4417 - val_d_loss: 0.5960 - val_v_ccc: 0.4686 - val_a_ccc: 0.5631 - val_d_ccc: 0.4108
Epoch 9/50
6295/6295 [

6295/6295 [==============================] - 69s 11ms/step - loss: 1.1303 - v_loss: 0.3685 - a_loss: 0.3364 - d_loss: 0.4252 - v_ccc: 0.6313 - a_ccc: 0.6638 - d_ccc: 0.5747 - val_loss: 1.6470 - val_v_loss: 0.5734 - val_a_loss: 0.4995 - val_d_loss: 0.5855 - val_v_ccc: 0.4261 - val_a_ccc: 0.5085 - val_d_ccc: 0.4184
Epoch 3/50
6295/6295 [==============================] - 69s 11ms/step - loss: 0.9737 - v_loss: 0.2811 - a_loss: 0.3105 - d_loss: 0.3822 - v_ccc: 0.7189 - a_ccc: 0.6894 - d_ccc: 0.6180 - val_loss: 1.5906 - val_v_loss: 0.6382 - val_a_loss: 0.4543 - val_d_loss: 0.5100 - val_v_ccc: 0.3694 - val_a_ccc: 0.5486 - val_d_ccc: 0.4915
Epoch 4/50
6295/6295 [==============================] - 68s 11ms/step - loss: 0.8799 - v_loss: 0.2406 - a_loss: 0.2909 - d_loss: 0.3484 - v_ccc: 0.7592 - a_ccc: 0.7092 - d_ccc: 0.6516 - val_loss: 1.4746 - val_v_loss: 0.5389 - val_a_loss: 0.4199 - val_d_loss: 0.5278 - val_v_ccc: 0.4637 - val_a_ccc: 0.5846 - val_d_ccc: 0.4771
Epoch 5/50
6295/6295 [===========

Epoch 13/50
6295/6295 [==============================] - 70s 11ms/step - loss: 0.5062 - v_loss: 0.1347 - a_loss: 0.1805 - d_loss: 0.1907 - v_ccc: 0.8652 - a_ccc: 0.8194 - d_ccc: 0.8092 - val_loss: 1.5906 - val_v_loss: 0.5604 - val_a_loss: 0.4605 - val_d_loss: 0.5809 - val_v_ccc: 0.4462 - val_a_ccc: 0.5403 - val_d_ccc: 0.4230
Epoch 14/50
6295/6295 [==============================] - 70s 11ms/step - loss: 0.4804 - v_loss: 0.1255 - a_loss: 0.1699 - d_loss: 0.1849 - v_ccc: 0.8745 - a_ccc: 0.8301 - d_ccc: 0.8151 - val_loss: 1.6029 - val_v_loss: 0.5443 - val_a_loss: 0.4885 - val_d_loss: 0.5859 - val_v_ccc: 0.4666 - val_a_ccc: 0.5118 - val_d_ccc: 0.4187
Epoch 15/50
2170/2170 [==============================] - 5s 3ms/step
[0.37872588634490967, 0.557071328163147, 0.42261943221092224]
4
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 80s 13ms/step - loss: 2.3135 - v_loss: 0.8317 - a_loss: 0.7090 - d_loss: 0.7718 - v_ccc: 0.1677 - a_ccc: 0.29

Epoch 23/50
2170/2170 [==============================] - 6s 3ms/step
[0.4070887863636017, 0.5622376799583435, 0.44853848218917847]
5
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 75s 12ms/step - loss: 2.2512 - v_loss: 0.8260 - a_loss: 0.6813 - d_loss: 0.7425 - v_ccc: 0.1733 - a_ccc: 0.3181 - d_ccc: 0.2574 - val_loss: 2.1377 - val_v_loss: 0.7614 - val_a_loss: 0.7261 - val_d_loss: 0.6674 - val_v_ccc: 0.2455 - val_a_ccc: 0.2770 - val_d_ccc: 0.3398
Epoch 2/50
6295/6295 [==============================] - 70s 11ms/step - loss: 1.2045 - v_loss: 0.4243 - a_loss: 0.3404 - d_loss: 0.4393 - v_ccc: 0.5757 - a_ccc: 0.6594 - d_ccc: 0.5605 - val_loss: 1.7128 - val_v_loss: 0.6333 - val_a_loss: 0.5183 - val_d_loss: 0.5767 - val_v_ccc: 0.3753 - val_a_ccc: 0.4870 - val_d_ccc: 0.4249
Epoch 3/50
6295/6295 [==============================] - 70s 11ms/step - loss: 1.0472 - v_loss: 0.3335 - a_loss: 0.3132 - d_loss: 0.4006 - v_ccc: 0.6663 - a_ccc: 0.6870

...

6295/6295 [==============================] - 86s 14ms/step - loss: 0.6756 - v_loss: 0.1772 - a_loss: 0.2331 - d_loss: 0.2651 - v_ccc: 0.8227 - a_ccc: 0.7669 - d_ccc: 0.7349 - val_loss: 1.5414 - val_v_loss: 0.5629 - val_a_loss: 0.4280 - val_d_loss: 0.5684 - val_v_ccc: 0.4515 - val_a_ccc: 0.5710 - val_d_ccc: 0.4360
Epoch 9/50
6295/6295 [==============================] - 87s 14ms/step - loss: 0.6281 - v_loss: 0.1670 - a_loss: 0.2155 - d_loss: 0.2458 - v_ccc: 0.8331 - a_ccc: 0.7843 - d_ccc: 0.7545 - val_loss: 1.5474 - val_v_loss: 0.5579 - val_a_loss: 0.4377 - val_d_loss: 0.5681 - val_v_ccc: 0.4505 - val_a_ccc: 0.5644 - val_d_ccc: 0.4377
Epoch 10/50
6295/6295 [==============================] - 85s 14ms/step - loss: 0.5955 - v_loss: 0.1565 - a_loss: 0.2073 - d_loss: 0.2319 - v_ccc: 0.8436 - a_ccc: 0.7927 - d_ccc: 0.7683 - val_loss: 1.5380 - val_v_loss: 0.5292 - val_a_loss: 0.4512 - val_d_loss: 0.5760 - val_v_ccc: 0.4799 - val_a_ccc: 0.5531 - val_d_ccc: 0.4291
Epoch 11/50
6295/6295 [=========

Epoch 13/50
6295/6295 [==============================] - 84s 13ms/step - loss: 0.4973 - v_loss: 0.1316 - a_loss: 0.1777 - d_loss: 0.1884 - v_ccc: 0.8686 - a_ccc: 0.8223 - d_ccc: 0.8118 - val_loss: 1.5801 - val_v_loss: 0.5279 - val_a_loss: 0.4574 - val_d_loss: 0.6069 - val_v_ccc: 0.4734 - val_a_ccc: 0.5467 - val_d_ccc: 0.3998
Epoch 14/50
6295/6295 [==============================] - 84s 13ms/step - loss: 0.4754 - v_loss: 0.1241 - a_loss: 0.1699 - d_loss: 0.1817 - v_ccc: 0.8759 - a_ccc: 0.8302 - d_ccc: 0.8184 - val_loss: 1.6010 - val_v_loss: 0.5454 - val_a_loss: 0.4705 - val_d_loss: 0.6041 - val_v_ccc: 0.4630 - val_a_ccc: 0.5344 - val_d_ccc: 0.4015
Epoch 15/50
6295/6295 [==============================] - 85s 13ms/step - loss: 0.4523 - v_loss: 0.1205 - a_loss: 0.1640 - d_loss: 0.1679 - v_ccc: 0.8796 - a_ccc: 0.8360 - d_ccc: 0.8321 - val_loss: 1.5813 - val_v_loss: 0.5257 - val_a_loss: 0.4785 - val_d_loss: 0.5975 - val_v_ccc: 0.4839 - val_a_ccc: 0.5264 - val_d_ccc: 0.4084
Epoch 16/50
6295/62

6295/6295 [==============================] - 87s 14ms/step - loss: 0.7964 - v_loss: 0.2246 - a_loss: 0.2629 - d_loss: 0.3091 - v_ccc: 0.7753 - a_ccc: 0.7372 - d_ccc: 0.6910 - val_loss: 1.5485 - val_v_loss: 0.5795 - val_a_loss: 0.4298 - val_d_loss: 0.5542 - val_v_ccc: 0.4277 - val_a_ccc: 0.5750 - val_d_ccc: 0.4489
Epoch 7/50
6295/6295 [==============================] - 86s 14ms/step - loss: 0.7429 - v_loss: 0.2087 - a_loss: 0.2464 - d_loss: 0.2877 - v_ccc: 0.7913 - a_ccc: 0.7535 - d_ccc: 0.7123 - val_loss: 1.5588 - val_v_loss: 0.5528 - val_a_loss: 0.4281 - val_d_loss: 0.5943 - val_v_ccc: 0.4568 - val_a_ccc: 0.5732 - val_d_ccc: 0.4111
Epoch 8/50
6295/6295 [==============================] - 85s 14ms/step - loss: 0.6962 - v_loss: 0.1910 - a_loss: 0.2342 - d_loss: 0.2710 - v_ccc: 0.8089 - a_ccc: 0.7658 - d_ccc: 0.7291 - val_loss: 1.5876 - val_v_loss: 0.5569 - val_a_loss: 0.4541 - val_d_loss: 0.5933 - val_v_ccc: 0.4497 - val_a_ccc: 0.5512 - val_d_ccc: 0.4115
Epoch 9/50
6295/6295 [===========

Epoch 17/50
2170/2170 [==============================] - 6s 3ms/step
[0.42618751525878906, 0.5622141361236572, 0.4529092311859131]
26
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 100s 16ms/step - loss: 2.4428 - v_loss: 0.8962 - a_loss: 0.7561 - d_loss: 0.7896 - v_ccc: 0.1037 - a_ccc: 0.2434 - d_ccc: 0.2102 - val_loss: 2.2162 - val_v_loss: 0.7800 - val_a_loss: 0.7126 - val_d_loss: 0.7340 - val_v_ccc: 0.2278 - val_a_ccc: 0.2874 - val_d_ccc: 0.2685
Epoch 2/50
6295/6295 [==============================] - 86s 14ms/step - loss: 1.2293 - v_loss: 0.4364 - a_loss: 0.3567 - d_loss: 0.4364 - v_ccc: 0.5638 - a_ccc: 0.6433 - d_ccc: 0.5636 - val_loss: 1.5868 - val_v_loss: 0.6001 - val_a_loss: 0.4763 - val_d_loss: 0.5204 - val_v_ccc: 0.4051 - val_a_ccc: 0.5281 - val_d_ccc: 0.4800
Epoch 3/50
6295/6295 [==============================] - 85s 14ms/step - loss: 1.0324 - v_loss: 0.3251 - a_loss: 0.3152 - d_loss: 0.3920 - v_ccc: 0.6748 - a_ccc: 0.68

6295/6295 [==============================] - 85s 13ms/step - loss: 0.5977 - v_loss: 0.1624 - a_loss: 0.2047 - d_loss: 0.2309 - v_ccc: 0.8376 - a_ccc: 0.7953 - d_ccc: 0.7695 - val_loss: 1.5715 - val_v_loss: 0.5473 - val_a_loss: 0.4768 - val_d_loss: 0.5561 - val_v_ccc: 0.4619 - val_a_ccc: 0.5229 - val_d_ccc: 0.4436
Epoch 11/50
6295/6295 [==============================] - 84s 13ms/step - loss: 0.5714 - v_loss: 0.1514 - a_loss: 0.1993 - d_loss: 0.2206 - v_ccc: 0.8485 - a_ccc: 0.8006 - d_ccc: 0.7795 - val_loss: 1.5249 - val_v_loss: 0.5391 - val_a_loss: 0.4389 - val_d_loss: 0.5609 - val_v_ccc: 0.4677 - val_a_ccc: 0.5659 - val_d_ccc: 0.4416
Epoch 12/50
6295/6295 [==============================] - 84s 13ms/step - loss: 0.5401 - v_loss: 0.1426 - a_loss: 0.1914 - d_loss: 0.2063 - v_ccc: 0.8574 - a_ccc: 0.8087 - d_ccc: 0.7938 - val_loss: 1.5692 - val_v_loss: 0.5618 - val_a_loss: 0.4576 - val_d_loss: 0.5644 - val_v_ccc: 0.4462 - val_a_ccc: 0.5462 - val_d_ccc: 0.4383
Epoch 13/50
2170/2170 [========